## Realiza a primeira coleta de informações sobre o vídeo

In [126]:
# Bibliotecas
import os
import pandas as pd
import sqlite3
from datetime import datetime
from googleapiclient.discovery import build

In [127]:
# Carrega API Key e inicializa cliente
api_key = os.getenv('YOUTUBE_API_KEY')
youtube = build('youtube', 'v3', developerKey=api_key)

In [128]:
# Conecta (ou cria) o banco SQLite local
conn = sqlite3.connect('../data/sql/videos.db')
cursor = conn.cursor()

In [129]:
# Cria a tabela se não existir
cursor.execute('''
    CREATE TABLE IF NOT EXISTS videos_info (
        video_id TEXT PRIMARY KEY,
        titulo TEXT,
        descricao TEXT,
        data_coleta TEXT,
        visualizacoes INTEGER,
        likes INTEGER,
        comentarios INTEGER
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS channels_info (
    channel_id TEXT PRIMARY KEY,
    nome TEXT,
    descricao TEXT,
    inscritos INTEGER,
    visualizacoes INTEGER,
    total_videos INTEGER,
    data_coleta TEXT
    )
''')

conn.commit()

In [130]:
# Variáveis locais
csv_path = "../config/videos_monitorados.csv"

In [131]:
# Lê os vídeos monitorados com status ativo e adiciona em uma variável de lista
df_videos = pd.read_csv(csv_path)
ids_ativos = df_videos[df_videos['status'] == 'ativo']['video_id'].tolist()

### EXTRAÇÃO

In [ ]:
# Extrair dados do vídeo 
# Itera pelos vídeos
for vid in ids_ativos:
    # Verifica se já existe no banco
    cursor.execute("SELECT 1 FROM videos_info WHERE video_id = ?", (vid,))
    if cursor.fetchone():
        print(f"Pulando banco já cadastrado")
        continue

    # Consulta API
    try:
        response = youtube.videos().list(
            part='snippet,statistics',
            id=vid
        ).execute()

        item = response['items'][0]
        snippet = item['snippet']
        stats = item['statistics']
        titulo = snippet.get('title', '')
        descricao = snippet.get('description', '')
        data_coleta = datetime.now().isoformat()
        visualizacoes = int(stats.get('viewCount', 0))
        likes = int(stats.get('likeCount', 0))
        comentarios = int(stats.get('commentCount', 0))
    except:
        print(f"Erro ao capturar os dados")
        
comentarios = int(stats.get('commentCount', 0))

# Insere os dados no banco
try:
    cursor.execute("""
        INSERT INTO videos_info (
            video_id, titulo, descricao, data_coleta,
            visualizacoes, likes, comentarios
        ) VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (vid, titulo, descricao, data_coleta, visualizacoes, likes, comentarios))
    print(f"{titulo} adicionado ao banco.")
except:
    pass

conn.commit()

Nenhum vídeo novo adicionado à lista
VINHETA SUPERCINE 2007 adicionado ao banco.


In [133]:
# Extrair informações de canal

canal_id = snippet.get('channelId', '')

# Verifica se o canal já tá no banco
cursor.execute("SELECT 1 FROM channels_info WHERE channel_id = ?", (canal_id,))
if cursor.fetchone():
    print(f"Canal {canal_id} já está no banco, pulando.")
else:
    # Consulta o canal na API
    canal_response = youtube.channels().list(
        part='snippet,statistics',
        id=canal_id
    ).execute()

    canal_info = canal_response['items'][0]
    c_snippet = canal_info['snippet']
    c_stats = canal_info['statistics']

    nome_canal = c_snippet.get('title', '')
    descricao_canal = c_snippet.get('description', '')
    inscritos = int(c_stats.get('subscriberCount', 0))
    visualizacoes = int(c_stats.get('viewCount', 0))
    qtd_videos = int(c_stats.get('videoCount', 0))
    data_coleta = datetime.now().isoformat()

    cursor.execute('''
        INSERT INTO channels_info (
            channel_id,
            nome,
            descricao,
            inscritos,
            visualizacoes,
            total_videos,
            data_coleta
        ) VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        canal_id,
        nome_canal,
        descricao_canal,
        inscritos,
        visualizacoes,
        qtd_videos,
        data_coleta
    ))

    conn.commit()
    print(f"Canal {nome_canal} inserido no banco.")

Canal PedroQuintelaify inserido no banco.
